> ¿Cuáles son los 5 códigos postales más comunes para las órdenes con estado ‘Refunded’? ¿Y cuál es el nombre más frecuente entre los clientes de esas direcciones?

Vamos a dividir el análisis en 2 partes para responder a esta consulta...

Primero vamos a filtrar las órdenes con estado 'Refunded' y agruparemos por código postal para encontrar si existen códigos postales más comunes entre estas órdenes. El código postal se encuentra al final de las direcciones... para este análisis vamos a considerar la shipping address ya que es la dirección a la que se envía el producto y por ende es la que se usa para los reembolsos.

Luego con toda la información de los códigos postales más comunes, vamos a buscar a los clientes que vivan en esos códigos postales usando la información de la tabla de clientes y observando su dirección (address). Luego vamos a agrupar por nombre para encontrar los nombres más comunes entre los clientes que vivan en esos códigos postales.

In [7]:
import pandas as pd

orders = pd.read_pickle("../../.data/clean/orders.pkl")
orders.info()

<class 'pandas.core.frame.DataFrame'>
Index: 4700000 entries, 1 to 4700000
Data columns (total 12 columns):
 #   Column            Dtype         
---  ------            -----         
 0   customer_id       Int64         
 1   order_date        datetime64[ns]
 2   status            category      
 3   payment_method    category      
 4   shipping_address  string        
 5   billing_address   string        
 6   discount_amount   Float64       
 7   shipping_cost     Float64       
 8   total_amount      Float64       
 9   currency          category      
 10  created_at        datetime64[ns]
 11  updated_at        datetime64[ns]
dtypes: Float64(3), Int64(1), category(3), datetime64[ns](3), string(2)
memory usage: 390.0+ MB


Primero entonces aplicaremos el filtrado por estado 'Refunded' y extraeremos el código postal de la dirección de envío (shipping_address).

In [8]:
import re

# Extract ZIP code from shipping_address for refunded orders
pattern_zip = re.compile(r"(\d{5})$")

refunded = orders[orders["status"] == "REFUNDED"].copy()
refunded["zip_shipping"] = refunded["shipping_address"].str.extract(pattern_zip)[0]

refunded["zip_shipping"] = refunded["zip_shipping"].astype("category")

refunded = refunded[["zip_shipping"]].reset_index()

refunded.head(10)

,order_id,zip_shipping
0,3,61083
1,77,38802
2,156,81233
3,260,21155
4,374,77293
5,480,87660
6,678,53020
7,882,67524
8,1301,58891
9,1473,84003


In [9]:
refunded.describe()

,order_id,zip_shipping
count,43249,39335
unique,43249,32425
top,3,70696
freq,1,6


Ahora con todos los pedidos, haremos el agrupado y conteo de los pedidos por código postal y analizaremos la distribución previo a encontrar los 5 códigos postales más comunes.

In [10]:
top_zip = refunded.groupby("zip_shipping", observed=True).agg(
    refunded_orders=(
        "order_id",
        "count",
    )
)

top_zip

,refunded_orders
zip_shipping,
00503,1
00505,1
00506,1
00507,1
00514,1
...,...
99941,1
99942,1
99943,1


In [11]:
top_zip.describe(percentiles=[0.1, 0.25, 0.5, 0.75, 0.9, 0.99])

,refunded_orders
count,32425.000000
mean,1.213107
std,0.474733
min,1.000000
10%,1.000000
25%,1.000000
50%,1.000000
75%,1.000000
90%,2.000000
99%,3.000000


En este caso se puede ver que el promedio de los 28k zips es 1.1 pedidos devueltos y el máximo es de 6 pedidos devueltos... esto significa que no hay códigos postales con una cantidad de pedidos devueltos muy alta. De hecho, si vemos los cuartiles vemos que hasta el 75% de los códigos postales tienen como mucho 1 pedido devuelto o menos.

Para aclarar como hicimos el filtrado solamente por los pedidos con estado 'Refunded', estaremos obviando todos los códigos postales que no tengan pedidos devueltos por ende la madia será aún más baja si consideramos todos los códigos postales.

Veremos entonces los 5 códigos postales más comunes entre los pedidos devueltos.

In [12]:
TOP_N_ZIPS = 5

sorted_zips_by_refunded_orders = top_zip.sort_values(
    by="refunded_orders", ascending=False
)

top_n_zips_by_refunded_orders = sorted_zips_by_refunded_orders.head(TOP_N_ZIPS)
top_n_zips_by_refunded_orders

,refunded_orders
zip_shipping,
70696,6
82319,5
47612,5
83755,5
11954,5


Ahora sabiendo cuáles son los 5 códigos postales más comunes entre los pedidos devueltos, buscaremos los clientes que vivan en esos códigos postales.

In [13]:
df_customers = pd.read_pickle("../../.data/clean/customers.pkl")
df_customers.info()

<class 'pandas.core.frame.DataFrame'>
Index: 500000 entries, 1 to 500000
Data columns (total 12 columns):
 #   Column             Non-Null Count   Dtype         
---  ------             --------------   -----         
 0   first_name         470197 non-null  string        
 1   date_of_birth      454914 non-null  datetime64[ns]
 2   gender             436521 non-null  category      
 3   country            470022 non-null  category      
 4   city               469805 non-null  category      
 5   postal_code        425000 non-null  string        
 6   address            470117 non-null  string        
 7   registration_date  455155 non-null  datetime64[ns]
 8   last_login         454954 non-null  datetime64[ns]
 9   is_active          500000 non-null  boolean       
 10  customer_segment   454954 non-null  string        
 11  marketing_consent  500000 non-null  boolean       
dtypes: boolean(2), category(3), datetime64[ns](3), string(4)
memory usage: 35.8 MB


In [14]:
df_customers.head()

,first_name,date_of_birth,gender,country,city,postal_code,address,registration_date,last_login,is_active,customer_segment,marketing_consent
customer_id,,,,,,,,,,,,
1,Kayla,1948-05-20,F,Brazil,South Michelle,70351,<NA>,2022-09-22 09:40:47.913063,2025-01-07 12:09:24.323425,True,Regular,True
2,Carolyn,1983-03-19,F,France,Lake Shaneville,65372,"247 Allison Overpass Suite 960 North Dillon, A...",2022-11-14 23:20:08.324356,2024-08-25 17:36:03.417619,True,Budget,False
3,Deborah,1940-05-23,M,Usa,Lisaborough,32855,"61358 Mills Spur Lake Tiffany, MS 52073",2022-10-14 15:49:17.728606,2023-04-10 06:15:20.412198,True,Budget,True
4,Rachel,1969-12-21,M,Australia,South Michaelborough,11530,undefined,2024-03-16 09:18:53.251566,2025-03-20 12:06:38.326651,True,<NA>,False
5,Lisa,1940-05-18,M,Japan,Herrerabury,61228,"6184 King Trail Lake Trevor, IA 55611",2023-07-21 10:07:22.968497,2025-04-19 11:40:01.441135,True,Regular,True


In [15]:
df_customers["zip"] = df_customers["address"].str.extract(pattern_zip)[0]

df_customers = df_customers[["first_name", "postal_code", "zip"]]

df_customers.head(10)

,first_name,postal_code,zip
customer_id,,,
1,Kayla,70351,<NA>
2,Carolyn,65372,64327
3,Deborah,32855,52073
4,Rachel,11530,<NA>
5,Lisa,61228,55611
6,Victoria,49605,15900
7,Caitlin,<NA>,01974
8,Elizabeth,94992,90700
9,Jane,18974,66020


Como podemos ver en el dataset tenemos un problema y es que los códigos postales y los zips no coinciden... probablemente esto se deba a inconsistencias en la carga de los usuarios... para ser consistentes usaremos el código postal que extrajimos de la dirección de los clientes (address) el zip y descartaremos el postal_code que viene en el dataset.

In [16]:
df_customers = df_customers[["first_name", "zip"]]

customers_in_top_zips = df_customers[df_customers["zip"].isin(
    top_n_zips_by_refunded_orders.index
)]

customers_in_top_zips.info()

<class 'pandas.core.frame.DataFrame'>
Index: 24 entries, 44 to 496917
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   first_name  22 non-null     string
 1   zip         24 non-null     string
dtypes: string(2)
memory usage: 576.0 bytes


Ahora si agrupamos por nombre (no apellido) y contamos la cantidad de clientes con ese nombre en los códigos postales más comunes entre los pedidos devueltos.

In [17]:
name_counts = customers_in_top_zips["first_name"].value_counts()

name_counts.describe()

count        21.0
mean     1.047619
std      0.218218
min           1.0
25%           1.0
50%           1.0
75%           1.0
max           2.0
Name: count, dtype: Float64

In [18]:
TOP_N_CUSTOMER_NAMES = 10

sorted_names = name_counts.sort_values(ascending=False)
sorted_names.head(TOP_N_CUSTOMER_NAMES)

first_name
Brandon     2
Lori        1
Michelle    1
Kendra      1
Jennifer    1
Matthew     1
Tonya       1
Connie      1
Jessica     1
Russell     1
Name: count, dtype: Int64

Si bien "Brandon" resulta el nombre de cliente más común, solo hay 2 clientes con ese nombre en los códigos postales más comunes entre los pedidos devueltos. Esto nos lleva a pensar que no existe tal cosa como nombres más frecuentes entre los clientes que vivan en los 5 códigos postales más comunes entre los pedidos devueltos.

Esto tiene sentido ya que no hay demasiados clientes en esos códigos postales, por los que vimos antes, tan solo 24 clientes en total con 21 nombres diferentes (2 clientes llamados "Brandon" y 2 nulos).